Project to perform RFM Analysis on sales dataset

Import packages

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
pd.options.display.float_format = '{:20.2f}'.format
pd.set_option('display.max_columns', 999)

Read in the csv data file and check loaded correctly

In [ ]:
df = pd.read_csv("C:\\Users\\alexd\\Python Projects\\k_means/online_retail_II_p1.csv", encoding='ISO-8859-1')
df.head()


In [ ]:
df.describe()

problems observed above - Qty and Price both have neg mins - check these
number of customer ids is many less than the qty/price count

In [ ]:
df.describe(include='O')

problems from above - unique stock codes <> description

In [ ]:
# check Customer ID for NaN - this are of no value and will be dropped
df[df['Customer ID'].isna()].head(15)

In [ ]:
# Check for neg Qty - 'C' means that the order was cancelled or returned - these will be excluded
df[df['Quantity'] < 0].head(15)

In [ ]:
# Cast the Invoice column to string and check for other alpha leaders
# Use REGEX to find the leaders ^ and $ as anchors //d for digits
df['Invoice'] = df['Invoice'].astype("str")
df[df['Invoice'].str.match("^\\d{6}$") == False]


In [ ]:
# Find how what alpha leaders there are. Use REGEX to scrub off numbers and show unique what's left
df["Invoice"].str.replace("[0-9]", "", regex=True).unique()

In [ ]:
# We know 'C' is cancel, check what 'A' denotes
# A denote bad debt so both C and A will be dropped
df[df["Invoice"].str.startswith("A")]

In [ ]:
# Check the StockCodes for interesting things
# Cast StockCode as string to start
# Use REGEX to find anomalies to the StockCode that should be 5 digits
df['StockCode'] = df['StockCode'].astype('str')
df[(df['StockCode'].str.match("^\\d{5}$") == False) & (df['StockCode'].str.match("^\\d{5}[a-zA-Z]+$") == False)]["StockCode"].unique()

There's pile of non-conforming StockCodes. Each has to be investigated.
The code line below is the iterative process of keep/drop

In [ ]:
df[df["StockCode"].str.contains("DOT")]
# DOT is of no value and would be dropped
# after going through the list only PADS is going to be kept, the rest dropped

In [ ]:
df[df["StockCode"].str.contains("PADS")]

Clean up the data

In [ ]:
cleaned_df = df.copy()


Clean up the Invoice column

In [ ]:
cleaned_df['Invoice'] = cleaned_df['Invoice'].astype('str')

mask = (
    cleaned_df['Invoice'].str.match("^\\d{6}$") == True   
)
cleaned_df = cleaned_df[mask]
cleaned_df

Clean the StockCode column
Three matches to keep 1. 5 digit stock codes 2. 5 digit stock codes with alpha follower 3. StockCode = PADS

In [ ]:
cleaned_df['StockCode'] = cleaned_df['StockCode'].astype('str')

mask = (
    (cleaned_df['StockCode'].str.match("^\\d{5}$") == True)
    | (cleaned_df["StockCode"].str.match("^\\d{5}[a-zA-Z]+$") == True)
    | (cleaned_df['StockCode'].str.match("^PADS$") == True)
)

cleaned_df = cleaned_df[mask]
cleaned_df

Drop the lines w/ Customer ID = NaN
After dropping the Customer ID NaNs the counts should line up Price/Qty/Customer ID

In [ ]:
cleaned_df.dropna(subset=['Customer ID'], inplace=True)
cleaned_df.describe()

In [ ]:
# Clean up Price where price = 0
# How many 0 entries - 28
len(cleaned_df[cleaned_df["Price"] == 0])

In [ ]:
cleaned_df = cleaned_df[cleaned_df["Price"] > 0.0]
len(cleaned_df[cleaned_df["Price"] == 0])  ## 0 entries w/ price = 0

How much data left/lost in data cleanup
77% is left so 23% of data lost during cleanup

In [ ]:
len(cleaned_df) / len(df)

Aggregate the data into RFM features

Extend the Price and Quantity into Sales Total

In [ ]:
cleaned_df['SalesLineTotal'] = cleaned_df['Quantity'] * cleaned_df['Price']
cleaned_df

Create new df for aggregated data

In [ ]:
aggregated_df = cleaned_df.groupby(by='Customer ID', as_index=False) \
    .agg(
        MonetaryValue=("SalesLineTotal", "sum"),
        Frequency= ("Invoice", "nunique"),
        LastInvoiceDate=("InvoiceDate", "max")
    )
    
aggregated_df.head(10)

Add Recency by using max date and lastest date of invoice
Set LastInvoiceDate to type datetime

In [ ]:
aggregated_df['LastInvoiceDate'] = pd.to_datetime(aggregated_df['LastInvoiceDate'])
max_invoice_date = aggregated_df["LastInvoiceDate"].max()
aggregated_df["Recency"] = (max_invoice_date - aggregated_df['LastInvoiceDate']).dt.days

aggregated_df

Visualize with histograms and box plots

In [ ]:
plt.figure(figsize=( 15, 5))

plt.subplot(1, 3, 1)
plt.hist(aggregated_df['MonetaryValue'], bins=10, color='lightgreen', edgecolor='black')
plt.title('Monetary Value Distribution')
plt.xlabel(' Monetary Value')
plt.ylabel('Count')

plt.subplot(1, 3, 2)
plt.hist(aggregated_df['Frequency'], bins=10, color='skyblue', edgecolor='black')
plt.title('Frequency Distribution')
plt.xlabel(' Frequency')
plt.ylabel('Count')

plt.subplot(1, 3, 3)
plt.hist(aggregated_df['Recency'], bins=10, color='salmon', edgecolor='black')
plt.title('Recency Distribution')
plt.xlabel(' Recency')
plt.ylabel('Count')

plt.tight_layout()
plt.show()





In [ ]:
plt.figure(figsize=( 15, 5))

plt.subplot(1, 3, 1)
sns.boxplot(data=aggregated_df['MonetaryValue'], color='lightgreen')
plt.title('Monetary Value Boxplot')
plt.xlabel(' Monetary Value')
plt.ylabel('Count')

plt.subplot(1, 3, 2)
sns.boxplot(data=aggregated_df['Frequency'], color='skyblue')
plt.title('Frequency Boxplot')
plt.xlabel(' Frequency')
plt.ylabel('Count')

plt.subplot(1, 3, 3)
sns.boxplot(data=aggregated_df['Recency'], color='salmon')
plt.title('Recency Boxplot')
plt.xlabel(' Recency')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

The histogram shows little distribution with so much of the data at the bottom of the plot.
This is confirmed by the boxplot that shows the large number of big outliers in the MV and Freq columns
These outliers are important to the data set so they will be removed but saved to process on their own

monetary_outliers_df = aggregated_df[(aggregated_df["MonetaryValue"] > (M_Q3 + 1.5 * M_IQR)) | (aggregated_df["MonetaryValue"] < (M_Q1 - 1.5 * M_IQR))].copy()

monetary_outliers_df.describe()

In [ ]:
# Remove the Monetary outliers = 423

M_Q1 = aggregated_df['MonetaryValue'].quantile(0.25)
M_Q3 = aggregated_df['MonetaryValue'].quantile(0.75)
M_IQR = M_Q3 - M_Q1
monetary_outliers_df = aggregated_df[(aggregated_df['MonetaryValue'] > (M_Q3 + 1.5 * M_IQR)) 
                                     | (aggregated_df['MonetaryValue'] < (M_Q1 - 1.5 * M_IQR))].copy()

monetary_outliers_df.describe()

In [ ]:
# Remove the Frequency outliers = 279

F_Q1 = aggregated_df['Frequency'].quantile(0.25)
F_Q3 = aggregated_df['Frequency'].quantile(0.75)
F_IQR = F_Q3 -F_Q1
frequency_outliers_df = aggregated_df[(aggregated_df['Frequency'] > (F_Q3 + 1.5 * F_IQR)) 
                                     | (aggregated_df['Frequency'] < (F_Q1 - 1.5 * F_IQR))].copy()

frequency_outliers_df.describe()

Now take the outliers out of the aggregated_df
Create a new df and use ~ meaning 'not' from the two outlier df's using their indicies

In [ ]:
non_outliers_df = aggregated_df[(~aggregated_df.index.isin(monetary_outliers_df.index)) & 
                                (~aggregated_df.index.isin(frequency_outliers_df.index))]

non_outliers_df.describe()

In [ ]:
# Redo the boxplots - they do show the difference once the extreme number of outliers are removed

plt.figure(figsize=( 15, 5))

plt.subplot(1, 3, 1)
sns.boxplot(data=non_outliers_df['MonetaryValue'], color='lightgreen')
plt.title('Monetary Value Boxplot')
plt.xlabel(' Monetary Value')
plt.ylabel('Count')

plt.subplot(1, 3, 2)
sns.boxplot(data=non_outliers_df['Frequency'], color='skyblue')
plt.title('Frequency Boxplot')
plt.xlabel(' Frequency')
plt.ylabel('Count')

plt.subplot(1, 3, 3)
sns.boxplot(data=non_outliers_df['Recency'], color='salmon')
plt.title('Recency Boxplot')
plt.xlabel(' Recency')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

In [ ]:
# 3-D Scatterplot of Customer Data

fig = plt.figure(figsize=(8, 8))

ax = fig.add_subplot(projection="3d")

scatter = ax.scatter(non_outliers_df['MonetaryValue'], non_outliers_df['Frequency'], non_outliers_df['Recency'])

ax.set_xlabel('Monetary Value')
ax.set_ylabel('Frequency')
ax.set_zlabel('Recency')

ax.set_title('3_D Scatter Plot of Customer Data')

plt.show()

Data Scaling

The 3-D plot shows the data distribution but the axes show that the data is on 3 different scales each seperated by an order of magnitude - 10/100/1000
To properly work the data it needs to be scaled so that the each data axis is on a common scale
Standard Scaler forces all data onto a scale to a mean of 0 and a std dev of 1

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_data = scaler.fit_transform(non_outliers_df[['MonetaryValue', 'Frequency', 'Recency']])
scaled_data

In [ ]:
# scaled_data is returned as a numpy array and it needs to be returned to new df
scaled_data_df = pd.DataFrame(scaled_data, index=non_outliers_df.index,
                columns=("MonetaryValue", "Frequency", "Recency"))
scaled_data_df


In [ ]:
# New 3-D Scatterplot of Scaled Customer Data
fig = plt.figure(figsize=(8, 8))

ax = fig.add_subplot(projection="3d")

scatter = ax.scatter(scaled_data_df['MonetaryValue'], scaled_data_df['Frequency'], scaled_data_df['Recency'])

ax.set_xlabel('Monetary Value')
ax.set_ylabel('Frequency')
ax.set_zlabel('Recency')

ax.set_title('3_D Scatter Plot of Scaled Customer Data')

plt.show()


KMeans Clustering
First determine the number of clusters by letting the method run on k clusters and capture inertia
Determine the most likely number of clusters using the 'Elbow' method
Confirm choice using silhouette scores

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

max_k =12

inertia = []
silhouette_scores = []
k_values = range(2, max_k + 1) # work from 2 - 12 clusters

for k in k_values:
    kmeans = KMeans(n_clusters=k, n_init=10, random_state=42, max_iter=1000)
    cluster_labels = kmeans.fit_predict(scaled_data_df)
    sil_score = silhouette_score(scaled_data_df, cluster_labels)
    silhouette_scores.append(sil_score)
    inertia.append(kmeans.inertia_)
    
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.plot(k_values, inertia, marker='o')
plt.title('KMeans Inertia for Different Values of k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.xticks(k_values)
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(k_values, silhouette_scores, marker='o', color='orange')
plt.title('Silhouette Scores for Different Values of k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Silhouette Score')
plt.xticks(k_values)
plt.grid(True)

plt.tight_layout()
plt.show()


The KMeans test and the silhouette score suggests that 4 clusters would be the most useful for this data

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=42, max_iter=1000)
cluster_labels = kmeans.fit_predict(scaled_data_df)
cluster_labels

Add cluster labels to non_outlier_df for plotting

In [ ]:
non_outliers_df['Cluster'] = cluster_labels
non_outliers_df

In [ ]:
# Plot 3-D of cluster id'd by color map
cluster_colors = {0: '#1f77b4', #Blue
                  1: '#ff7f0e', # Orange
                  2: '#2ca02c', # Green
                  3: '#d62728', # Red
                  }

colors = non_outliers_df['Cluster'].map(cluster_colors)

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(projection='3d')

scatter = ax.scatter(non_outliers_df['MonetaryValue'],
                     non_outliers_df['Frequency'],
                     non_outliers_df['Recency'],
                     c=colors,
                     marker='o')

ax.set_xlabel('Monetary Value')
ax.set_ylabel('Frequency')
ax.set_zlabel('Recency')

ax.set_title(' 3D Scatterplot of Customer Data by Cluster')

plt.show()

Create Violin plots of the clusters to determine placing in features
The gray violin plot is the non-scaled data

In [ ]:
plt.figure(figsize=(12, 18))

plt.subplot(3, 1, 1)
sns.violinplot(x=non_outliers_df['Cluster'], y=non_outliers_df['MonetaryValue'], palette=cluster_colors, 
             hue=non_outliers_df['Cluster'])
sns.violinplot(y=non_outliers_df['MonetaryValue'], color='gray', linewidth=1.0)
plt.title('Monetary Value by Cluster')
plt.ylabel('Monetary Value')

plt.subplot(3, 1, 2)
sns.violinplot(x=non_outliers_df['Cluster'], y=non_outliers_df['Frequency'], palette=cluster_colors, 
             hue=non_outliers_df['Cluster'])
sns.violinplot(y=non_outliers_df['Frequency'], color='gray', linewidth=1.0)
plt.title('Frequency by Cluster')
plt.ylabel('Frequency')

plt.subplot(3, 1, 3)
sns.violinplot(x=non_outliers_df['Cluster'], y=non_outliers_df['Recency'], palette=cluster_colors, 
             hue=non_outliers_df['Cluster'])
sns.violinplot(y=non_outliers_df['Recency'], color='gray', linewidth=1.0)
plt.title('Recency by Cluster')
plt.ylabel('Recency')

plt.tight_layout()
plt.show()



Cluster 0 = Blue - high value and buy often - work to retain this cluster
Cluster 1 = Orange - low value/frequency but recent buyers - work to keep them buying
Cluster 2 = Green - least active and low value somewhat recent - work to get them back
Cluster 3 = Red - highest value and frequent buyers but not recent - work to keep them

Process the outliers from their dfs into one df to visualize their features
Do this by defining a variable that captures all of the overlap indicies using index.intersection 
Next drop the indicies in the variable from the two outlier dfs
Then concatenating the three elements into a new df
Manually assign clusters to each outlier feature

In [ ]:
# Isolate the overlap indicies and drop from existing dfs
overlap_indicies = monetary_outliers_df.index.intersection(frequency_outliers_df.index)

frequency_only_outliers = frequency_outliers_df.drop(overlap_indicies)
monetary_only_outliers = monetary_outliers_df.drop(overlap_indicies)
monetary_and_frequency_outliers = monetary_outliers_df.loc[overlap_indicies]

monetary_only_outliers['Cluster'] = -1
frequency_only_outliers['Cluster'] = -2
monetary_and_frequency_outliers['Cluster'] = -3

outlier_clusters_df = pd.concat([monetary_only_outliers, frequency_only_outliers, 
                                 monetary_and_frequency_outliers])

outlier_clusters_df

Visualize the features using the assigned clusters and violin plots

In [ ]:
cluster_colors = { -1: '#9467bd',
                  -2: '#8c564b', 
                  -3: '#e377c2'
                  }


plt.figure(figsize=(12, 18))

plt.subplot(3, 1, 1)
sns.violinplot(x=outlier_clusters_df['Cluster'], y=outlier_clusters_df['MonetaryValue'], palette=cluster_colors, 
             hue=outlier_clusters_df['Cluster'])
sns.violinplot(y=outlier_clusters_df['MonetaryValue'], color='gray', linewidth=1.0)
plt.title('Monetary Value by Cluster')
plt.ylabel('Monetary Value')

plt.subplot(3, 1, 2)
sns.violinplot(x=outlier_clusters_df['Cluster'], y=outlier_clusters_df['Frequency'], palette=cluster_colors, 
             hue=outlier_clusters_df['Cluster'])
sns.violinplot(y=outlier_clusters_df['Frequency'], color='gray', linewidth=1.0)
plt.title('Frequency by Cluster')
plt.ylabel('Frequency')

plt.subplot(3, 1, 3)
sns.violinplot(x=outlier_clusters_df['Cluster'], y=outlier_clusters_df['Recency'], palette=cluster_colors, 
             hue=outlier_clusters_df['Cluster'])
sns.violinplot(y=outlier_clusters_df['Recency'], color='gray', linewidth=1.0)
plt.title('Recency by Cluster')
plt.ylabel('Recency')


Cluster -1 Monetary outliers - hi $ but low frequency - work on keeping them and encourage purchases
Cluster - 2 Frequency outliers - lower $ but more frequent purchases - try to increase value spending
Cluster -3 Monetary and Frequency outliers - hi $ and hi frequency - develop strategies to keep them engaged

In [ ]:
cluster_labels = {
    0: "0-Retain",
    1: "1-Re-engage",
    2: "2-Nurture",
    3: "3-Reward",
    -1: "OM-Pamper",
    -2: "OF-Upsell",
    -3: "OMF-Delight"
}

Combine both outlier and non_outlier dfs together into a full clustering df

In [ ]:
full_clustering_df = pd.concat([non_outliers_df, outlier_clusters_df])
full_clustering_df

In [ ]:
# create a column and assign cluster_labels by mapping label to cluster
full_clustering_df["ClusterLabel"] = full_clustering_df["Cluster"].map(cluster_labels)

full_clustering_df

Visualize the complete data

In [ ]:
cluster_counts = full_clustering_df['ClusterLabel'].value_counts()

full_clustering_df["MonetaryValue per 100 pounds"] = full_clustering_df["MonetaryValue"] / 100.00
feature_means = full_clustering_df.groupby('ClusterLabel')[['Recency', 'Frequency', 'MonetaryValue per 100 pounds']].mean()

fig, ax1 = plt.subplots(figsize=(12, 8))

sns.barplot(x=cluster_counts.index, y=cluster_counts.values, ax=ax1, palette='viridis', hue=cluster_counts.index)
ax1.set_ylabel('Number of Customers', color='b')
ax1.set_title('Cluster Distribution with Average Feature Values')

ax2 = ax1.twinx()

sns.lineplot(data=feature_means, ax=ax2, palette='Set2', marker='o')
ax2.set_ylabel('Average Value', color='g')

plt.show()

In [ ]:
aggregated_df.to_csv("C:\\Users\\alexd\\Python Projects\\k_means/aggregated_data.csv", index=False, encoding='ISO-8859-1')
full_clustering_df.to_csv("C:\\Users\\alexd\\Python Projects\\k_means/full_clustering.csv", index=False, encoding='ISO-8859-1')
